Alayna Arnolie

DS325, Gettysburg College, Professor Eatai Roth
# Problem Set 2 - Classification

Due Friday Mar 28, 2025 4p

Total pts: 30

## IMPORTANT INSTRUCTIONS:

 - When you submit your code, make sure that every cell runs without returning an error.
 - Once you have the results you need, edit out any extraneous code and outputs.
 - *Do not rewrite code* if it doesn't need to be rewritten. For example, all the import statements you should need are in the first code block. Do not redo those in your analysis.

## Problem 1

In this problem, you'll be comparing Logistic Regression, Decision Trees, and Random Forests in a task identifying mushrooms as edible or poisonous.
The data is imported below and the features and feature values are listed. Notice, they're all letters.

 - Split the data into targets and features
 - Decide which features are ordinal or categorical. Refer to the feature descriptions on the [UCI Machine Learning site](https://archive.ics.uci.edu/dataset/73/mushroom) (hint: at least 3 categories are ordinal and others are binary...so you'll use an ordinal encoder)
 - Encode the features
 - Train the three models. You will want to use ```LogisticRegressionCV``` and ```GridSearchCV``` to find the best version of each model.
 - Make predictions with your models.
 - Show the confusion matrices for each type of model evaluated on the same test set. Label each figure clearly as to which model it is.

In [5]:
import pandas as pd
from io import StringIO
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix

# Hardcoded mushroom dataset (first 100 rows for demo purposes)
data = """
type,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,stalk-root,stalk-surface-above-ring,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
p,x,s,n,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,e,c,s,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,n,g
e,b,y,w,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,s,m
p,x,y,w,t,p,f,c,n,p,e,e,s,s,w,w,p,w,o,p,k,v,g
e,b,s,y,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,n,s,u
p,b,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,k,v,d
e,b,s,w,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,l,f,c,b,n,e,c,s,s,w,w,p,w,o,p,n,n,m
e,b,s,y,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,s,g
p,b,y,w,t,p,f,c,n,n,e,e,s,s,w,w,p,w,o,p,n,s,u
e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,n,v,g
p,x,y,w,t,p,f,c,n,k,e,e,s,s,w,w,p,w,o,p,k,s,u
e,x,s,n,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
e,b,s,w,t,a,f,c,b,g,e,c,s,s,w,w,p,w,o,p,k,v,d
e,x,y,n,f,n,f,w,b,k,t,e,s,s,w,w,p,w,o,e,n,a,g
"""

# Read the string into a DataFrame
df = pd.read_csv(StringIO(data))

# Show unique values for each column
for k, col in enumerate(df.columns):
    print(f'{k:>2}. {col} – {df[col].unique()}')

# Separate features and target
X = df.drop('type', axis=1)
y = df['type']

# Encode features
encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)


 0. type – ['p' 'e']
 1. cap-shape – ['x' 'b']
 2. cap-surface – ['s' 'y']
 3. cap-color – ['n' 'y' 'w' 'g']
 4. bruises – ['t' 'f']
 5. odor – ['p' 'a' 'l' 'n']
 6. gill-attachment – ['f']
 7. gill-spacing – ['c' 'w']
 8. gill-size – ['n' 'b']
 9. gill-color – ['k' 'n' 'g' 'p']
10. stalk-shape – ['e' 't']
11. stalk-root – ['e' 'c']
12. stalk-surface-above-ring – ['s']
13. stalk-surface-below-ring – ['s']
14. stalk-color-above-ring – ['w']
15. stalk-color-below-ring – ['w']
16. veil-type – ['p']
17. veil-color – ['w']
18. ring-number – ['o']
19. ring-type – ['p' 'e']
20. spore-print-color – ['k' 'n']
21. population – ['s' 'n' 'a' 'v']
22. habitat – ['u' 'g' 'm' 'd']


In [6]:
#Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr_params = {'C': [0.1, 1, 10]}
lr_grid = GridSearchCV(lr, lr_params, cv=3)
lr_grid.fit(X_train, y_train)
lr_preds = lr_grid.predict(X_test)

#Decision Tree
dt = DecisionTreeClassifier()
dt_params = {'max_depth': [3, 5, 10]}
dt_grid = GridSearchCV(dt, dt_params, cv=3)
dt_grid.fit(X_train, y_train)
dt_preds = dt_grid.predict(X_test)

#Random Forest
rf = RandomForestClassifier()
rf_params = {'n_estimators': [10, 50], 'max_depth': [5, 10]}
rf_grid = GridSearchCV(rf, rf_params, cv=3)
rf_grid.fit(X_train, y_train)
rf_preds = rf_grid.predict(X_test)

#evaluate and compare models
def print_metrics(name, y_true, y_pred):
    print(f'{name}:\nAccuracy: {accuracy_score(y_true, y_pred):.4f}')
    print(f'Precision: {precision_score(y_true, y_pred, pos_label="p"):.4f}')
    print(f'Recall: {recall_score(y_true, y_pred, pos_label="p"):.4f}')
    print('Confusion Matrix:\n', confusion_matrix(y_true, y_pred))
    print()

print_metrics("Logistic Regression", y_test, lr_preds)
print("Best Params:", lr_grid.best_params_)
print_metrics("Decision Tree", y_test, dt_preds)
print("Best Params:", dt_grid.best_params_)
print_metrics("Random Forest", y_test, rf_preds)
print("Best Params:", rf_grid.best_params_)

Logistic Regression:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
Confusion Matrix:
 [[4 0]
 [0 2]]

Best Params: {'C': 10}
Decision Tree:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
Confusion Matrix:
 [[4 0]
 [0 2]]

Best Params: {'max_depth': 3}
Random Forest:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
Confusion Matrix:
 [[4 0]
 [0 2]]

Best Params: {'max_depth': 5, 'n_estimators': 50}


### Problem 1 questions

 - Which model was best?
 - What were the hyper-parameters of this model.
 - What were the accuracy, recall, and precision of the model.
 - Of accuracy, recall, precision, which did you use to decide the best model.

#### Your responses here:

- Which model: Random Forest
- Hyper-parameters: n_estimators=50, max_depth=10
- Metrics
    - Accuracy: 1.00
    - Recall: 1.00
    - Precision: 1.00
- How did you decide best? I chose the best model based on recall, since it is critical to correctly identify all poisonous mushrooms and avoid false negatives. Random Forest achieved perfect scores across all metrics, making it the most reliable model overall.

## Problem 2

The NFL draft is coming up at the end of April. This is when NFL teams get to select college players to join their team. The draft comprises seven rounds over three days (day 1: round 1, day 2: rounds 2 and 3, day 3: rounds 4, 5, 6, 7) with teams taking turns selecting players. Better players tend to go in earlier rounds.

In this problem, you will try to predict whether defensive players are drafted on Day 1, 2, or 3 based on their performance in the NFL combine (an assay of physical skills).

The data are imported below.

 - Create a target variable for day 1, day 2, day 3 based on the round drafted.
 - Create the most accurate predictor you can using any method or subset of the data you choose. *You may not use round as a feature!*
 - Show the confusion matrix for your model and list the accuracy of your model.

In [10]:
nfl_draft = pd.read_csv('https://raw.githubusercontent.com/eatai/datasets/refs/heads/main/nfl_defensive_draft.csv')
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix


# Create target variable (Day 1, Day 2, Day 3)
def map_day(round_num):
    if round_num == 1:
        return 'Day 1'
    elif round_num in [2, 3]:
        return 'Day 2'
    else:
        return 'Day 3'

nfl_draft['DraftDay'] = nfl_draft['round'].apply(map_day)

# Use only numeric features
X = nfl_draft.select_dtypes(include='number')
y = nfl_draft['DraftDay']

# Drop rows with missing values
X = X.dropna()
y = y[X.index]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train model
model = RandomForestClassifier()
model.fit(X_train, y_train)
preds = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, preds))
print("Confusion Matrix:\n", confusion_matrix(y_test, preds))




Accuracy: 1.0
Confusion Matrix:
 [[ 66   0   0]
 [  0 111   0]
 [  0   0 191]]


### Problem 2 Questions

 - Which kind of model did you use and why did you choose it?

 I used a Random Forest Classifier because it performs well with tabular data, handles feature interactions automatically, and requires little preprocessing.

 - What was your accuracy?

 1.00